In [109]:
import re;

class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value

    def __lt__(self, other):
        return self.value < other.value
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __eq__(self, other):
        return self.value == other.value
    
    def __str__(self):
        return self.suit

class Hand:
    def __init__(self, hand_string, card_values, hand_ranking, hand_ranking_regexes):
        self.hand_value = 0
        self.hand_type = ""
        self.rank = 0
        self.rank_times_bet = 0

        split = hand_string.split(" ")
        self.cards = self.create_cards(split[0], card_values)
        self.sorted_cards = sorted(self.cards, reverse=True)
        self.bet = int(split[1])

        self.check_hand(self.sorted_cards, hand_ranking, hand_ranking_regexes)

    def create_cards(self, cards_string, card_values):
        cards = []
        for card in cards_string:
            cards.append(Card(card, card_values[card]))
        return cards
        
    def get_cards_string(self, cards):
        return "".join([str(card) for card in cards])

    def check_hand(self, sorted_cards, hand_ranking, hand_ranking_regexes):
        cards_string = self.get_cards_string(sorted_cards)
        for hand_type in hand_ranking:
            regex = hand_ranking_regexes.get(hand_type)
            if regex and re.search(regex, cards_string):
                hand_value = hand_ranking[hand_type]
                if hand_value > self.hand_value:
                    self.hand_value = hand_value
                    self.hand_type = hand_type
    
    def check_rank_times_bet(self):
        self.rank_times_bet = self.rank * self.bet

    def __lt__(self, other):
        if self.hand_value < other.hand_value:
            return True
        elif self.hand_value == other.hand_value:
            for self_card, other_card in zip(self.cards, other.cards):
                if self_card < other_card:
                    return True
                elif self_card > other_card:
                    return False
        return False

    def __gt__(self, other):
        if self.hand_value > other.hand_value:
            return True
        elif self.hand_value == other.hand_value:
            for self_card, other_card in zip(self.cards, other.cards):
                if self_card > other_card:
                    return True
                elif self_card < other_card:
                    return False
        return False

    def __eq__(self, other):
        if self.hand_value != other.hand_value:
            return False
        for self_card, other_card in zip(self.cards, other.cards):
            if self_card != other_card:
                return False
        return True

    def __str__(self):
        cards_string = self.get_cards_string(self.cards)
        sorted_cards_string = self.get_cards_string(self.sorted_cards)
        return "{}\t{} {} \t\t- {} \tbet: \t{} \trank: \t{} \trank times bet: \t{}".format(cards_string, sorted_cards_string, self.hand_type, self.hand_value, self.bet, self.rank, self.rank_times_bet)

linesText1 = """AAAAA 7
AA8AA 6
23332 5
TTT98 4
23432 3
A23A4 2
23456 1""".split("\n")

linesText2 = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".split("\n")

with open('input.txt', 'r') as file:
    linesText3 = file.read().split("\n")

card_values = {"2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7,
               "8": 8, "9": 9, "T": 10, "J": 11, "Q": 12,
               "K": 13, "A": 14}

hand_ranking = {"high_card": 1,  "one_pair": 2, "two_pairs": 3, "three_of_a_kind": 4,
                "full_house": 5, "four_of_a_kind": 6, "five_of_a_kind": 7}

hand_ranking_reverse = {value: key for key, value in hand_ranking.items()}
hand_ranking = hand_ranking

all_card_values = "".join(card_values.keys())

hand_ranking_regexes = {
    "high_card": r"(?!.*(.).*\1)[23456789TJQKA]{5}",
    "one_pair": r"([23456789TJQKA])\1",
    "two_pairs": r"([23456789TJQKA]?([23456789TJQKA])\2{1}[23456789TJQKA]?([23456789TJQKA])\3{1}[23456789TJQKA]?)",
    "three_of_a_kind": r"([23456789TJQKA])\1{2}",
    "full_house": r"(([23456789TJQKA])\2{2}([23456789TJQKA])\3{1})|(([23456789TJQKA])\5{1}([23456789TJQKA])\6{2})",
    "four_of_a_kind": r"([23456789TJQKA])\1{3}",
    "five_of_a_kind": r"([23456789TJQKA])\1{4}"
}

print(hand_ranking_regexes)

test_hand = Hand("24294 2", card_values, hand_ranking, hand_ranking_regexes)
print (test_hand)

lines = linesText3
hands = []
for line in lines:
    hand = Hand(line, card_values, hand_ranking, hand_ranking_regexes)
    hands.append(hand)

sorted_hands = sorted(hands, reverse=True)
for rank, hand in enumerate(sorted_hands, start=1):
    hand.rank = len(sorted_hands) - rank + 1
    hand.check_rank_times_bet()
    print(hand)


rank_times_bet = [hand.rank_times_bet for hand in sorted_hands]
sum_rank_times_bet = sum(rank_times_bet)
print(sum_rank_times_bet)

{'high_card': '(?!.*(.).*\\1)[23456789TJQKA]{5}', 'one_pair': '([23456789TJQKA])\\1', 'two_pairs': '([23456789TJQKA]?([23456789TJQKA])\\2{1}[23456789TJQKA]?([23456789TJQKA])\\3{1}[23456789TJQKA]?)', 'three_of_a_kind': '([23456789TJQKA])\\1{2}', 'full_house': '(([23456789TJQKA])\\2{2}([23456789TJQKA])\\3{1})|(([23456789TJQKA])\\5{1}([23456789TJQKA])\\6{2})', 'four_of_a_kind': '([23456789TJQKA])\\1{3}', 'five_of_a_kind': '([23456789TJQKA])\\1{4}'}
24294	94422 two_pairs 		- 3 	bet: 	2 	rank: 	0 	rank times bet: 	0
JJJJJ	JJJJJ five_of_a_kind 		- 7 	bet: 	911 	rank: 	1000 	rank times bet: 	911000
AAAAJ	AAAAJ four_of_a_kind 		- 6 	bet: 	658 	rank: 	999 	rank times bet: 	657342
AAAA2	AAAA2 four_of_a_kind 		- 6 	bet: 	109 	rank: 	998 	rank times bet: 	108782
AAA8A	AAAA8 four_of_a_kind 		- 6 	bet: 	694 	rank: 	997 	rank times bet: 	691918
AAA3A	AAAA3 four_of_a_kind 		- 6 	bet: 	3 	rank: 	996 	rank times bet: 	2988
ATAAA	AAAAT four_of_a_kind 		- 6 	bet: 	681 	rank: 	995 	rank times bet: 	677595


Your puzzle answer was 253866470.

The first half of this puzzle is complete! It provides one gold star: *

--- Part Two ---
To make things a little more interesting, the Elf introduces one additional rule. Now, J cards are jokers - wildcards that can act like whatever card would make the hand the strongest type possible.

To balance this, J cards are now the weakest individual cards, weaker even than 2. The other cards stay in the same order: A, K, Q, T, 9, 8, 7, 6, 5, 4, 3, 2, J.

J cards can pretend to be whatever card is best for the purpose of determining hand type; for example, QJJQ2 is now considered four of a kind. However, for the purpose of breaking ties between two hands of the same type, J is always treated as J, not the card it's pretending to be: JKKK2 is weaker than QQQQ2 because J is weaker than Q.

Now, the above example goes very differently:

32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483
32T3K is still the only one pair; it doesn't contain any jokers, so its strength doesn't increase.
KK677 is now the only two pair, making it the second-weakest hand.
T55J5, KTJJT, and QQQJA are now all four of a kind! T55J5 gets rank 3, QQQJA gets rank 4, and KTJJT gets rank 5.
With the new joker rule, the total winnings in this example are 5905.

Using the new joker rule, find the rank of every hand in your set. What are the new total winnings?

In [129]:
! pip install termcolor

from termcolor import colored

  Obtaining dependency information for termcolor from https://files.pythonhosted.org/packages/d9/5f/8c716e47b3a50cbd7c146f45881e11d9414def768b7cd9c5e6650ec2a80a/termcolor-2.4.0-py3-none-any.whl.metadata



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: C:\Users\dkasprzyk\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [177]:
import re;

class Card:
    def __init__(self, suit, value):
        self.suit = suit
        self.value = value

    def __lt__(self, other):
        return self.value < other.value
    
    def __gt__(self, other):
        return self.value > other.value
    
    def __eq__(self, other):
        return self.value == other.value
    
    def __str__(self):
        return self.suit

class Hand:
    def __init__(self, hand_string, card_values, hand_ranking, hand_ranking_regexes):
        self.hand_value = 0
        self.hand_type = ""
        self.rank = 0
        self.rank_times_bet = 0
        self.final_match = None

        split = hand_string.split(" ")
        self.cards = self.create_cards(split[0], card_values)
        self.sorted_cards = sorted(self.cards, reverse=True)
        self.bet = int(split[1])

        self.check_hand(self.sorted_cards, hand_ranking, hand_ranking_regexes)

    def create_cards(self, cards_string, card_values):
        cards = []
        for card in cards_string:
            cards.append(Card(card, card_values[card]))
        return cards
        
    def get_cards_string(self, cards):
        return "".join([str(card) for card in cards])

    def check_hand(self, sorted_cards, hand_ranking, hand_ranking_regexes):
        cards_string = self.get_cards_string(sorted_cards)
        for hand_type, regex_categories in hand_ranking_regexes.items():
            for category, regex in regex_categories.items():
                match = re.search(regex, cards_string)
                if match:
                    hand_value = hand_ranking[hand_type]
                    if hand_value > self.hand_value:
                        self.hand_value = hand_value
                        self.hand_type = hand_type
                        self.final_match = match.group()  # Update the final match
    
    def check_rank_times_bet(self):
        self.rank_times_bet = self.rank * self.bet

    def __lt__(self, other):
        if self.hand_value < other.hand_value:
            return True
        elif self.hand_value == other.hand_value:
            for self_card, other_card in zip(self.cards, other.cards):
                if self_card < other_card:
                    return True
                elif self_card > other_card:
                    return False
        return False

    def __gt__(self, other):
        if self.hand_value > other.hand_value:
            return True
        elif self.hand_value == other.hand_value:
            for self_card, other_card in zip(self.cards, other.cards):
                if self_card > other_card:
                    return True
                elif self_card < other_card:
                    return False
        return False

    def __eq__(self, other):
        if self.hand_value != other.hand_value:
            return False
        for self_card, other_card in zip(self.cards, other.cards):
            if self_card != other_card:
                return False
        return True

    def __str__(self):
        cards_string = self.get_cards_string(self.cards)
        sorted_cards_string = self.get_cards_string(self.sorted_cards)

        # Colorize the matched pattern in green
        highlighted_sorted_cards_string = sorted_cards_string.replace(self.final_match, colored(self.final_match, 'green'))

        # Within the green matched pattern, replace 'J' with red colorized 'J'
        highlighted_sorted_cards_string = re.sub(f"(?<=\033\[32m){self.final_match}(?=\033\[0m)", lambda match: match.group().replace('J', colored('J', 'red')), highlighted_sorted_cards_string)

        return "{}\t{}\t{}\t{} \t- {} \tbet: \t{} \trank: \t{} \trank times bet: \t{}".format(
            cards_string,
            highlighted_sorted_cards_string,
            self.final_match,
            self.hand_type,
            self.hand_value,
            self.bet,
            self.rank,
            self.rank_times_bet)


card_values = { "J": 1, "2": 2, "3": 3, "4": 4, "5": 5, "6": 6, "7": 7,
               "8": 8, "9": 9, "T": 10, "Q": 11,
               "K": 12, "A": 13}

hand_ranking_regexes = {
    "high_card": {
        "pattern": r"(?!.*(.).*\1)[23456789TQKA]{5}"
    },
    "one_pair": {
        "regular": r"([23456789TQKA])\1{1}",
        "one_wildcard": r"[23456789TQKA]J"
    },
    "two_pairs": {
        "regular_two_pairs": r"([23456789TQKA])\1{1}[23456789TQKA]?([23456789TQKA])\2{1}"
    },
    "three_of_a_kind": {
        "regular": r"([23456789TQKA])\1{2}",
        "one_wildcard": r"([23456789TQKA])\1{1}[23456789TQKAJ]?[23456789TQKA]?J",
        "two_wildcards": r"[23456789TQKA]JJ"
    },
    "full_house": {
        "regular_full_house_3_2": r"([23456789TQKA])\1{2}([23456789TQKA])\2{1}",
        "regular_full_house_2_3": r"([23456789TJQKA])\1{1}([23456789TJQKA])\2{2}",
        "regular_full_house_2_3_one_wildcard": r"([23456789TQKA])\1{1}([23456789TQKA])\2{1}J",
        "regular_full_house_2_3_two_wildcards": r"([23456789TQKA])\1{1}[23456789TQKA]JJ"
    },
    "four_of_a_kind": {
        "regular": r"([23456789TQKA])\1{3}",
        "one_wildcard": r"([23456789TQKA])\1{2}[23456789TQKA]?J",
        "two_wildcards": r"([23456789TQKA])\1{1}[23456789TQKA]?JJ",
        "three_wildcards": r"[23456789TQKA]JJJ"
    },
    "five_of_a_kind": {
        "regular": r"([23456789TQKA])\1{4}",
        "one_wildcard": r"([23456789TQKA])\1{3}J",
        "two_wildcards": r"([23456789TQKA])\1{2}JJ",
        "three_wildcards": r"([23456789TQKA])\1{1}JJJ",
        "four_wildcards": r"[23456789TQKA]JJJJ",
        "five_wildcards": r"JJJJJ"
    }
}

print(hand_ranking_regexes)

linesText1 = """JJJJJ 7
AJJJJ 7
AAJJJ 7
AAAJJ 7
AAAAJ 7
AQQQJ 6
A33JJ 6
AA3JJ 6
AA74J 33
AAAAA 8""".split("\n")

linesText2 = """32T3K 765
T55J5 684
KK677 28
KTJJT 220
QQQJA 483""".split("\n")


with open('input.txt', 'r') as file:
    linesText3 = file.read().split("\n")

lines = linesText3
hands = []
for line in lines:
    hand = Hand(line, card_values, hand_ranking, hand_ranking_regexes)
    hands.append(hand)

sorted_hands = sorted(hands, reverse=True)
for rank, hand in enumerate(sorted_hands, start=1):
    hand.rank = len(sorted_hands) - rank + 1
    hand.check_rank_times_bet()
    print(hand)

rank_times_bet = [hand.rank_times_bet for hand in sorted_hands]
sum_rank_times_bet = sum(rank_times_bet)
print(sum_rank_times_bet)

{'high_card': {'pattern': '(?!.*(.).*\\1)[23456789TQKA]{5}'}, 'one_pair': {'regular': '([23456789TQKA])\\1{1}', 'one_wildcard': '[23456789TQKA]J'}, 'two_pairs': {'regular_two_pairs': '([23456789TQKA])\\1{1}[23456789TQKA]?([23456789TQKA])\\2{1}'}, 'three_of_a_kind': {'regular': '([23456789TQKA])\\1{2}', 'one_wildcard': '([23456789TQKA])\\1{1}[23456789TQKAJ]?[23456789TQKA]?J', 'two_wildcards': '[23456789TQKA]JJ'}, 'full_house': {'regular_full_house_3_2': '([23456789TQKA])\\1{2}([23456789TQKA])\\2{1}', 'regular_full_house_2_3': '([23456789TJQKA])\\1{1}([23456789TJQKA])\\2{2}', 'regular_full_house_2_3_one_wildcard': '([23456789TQKA])\\1{1}([23456789TQKA])\\2{1}J', 'regular_full_house_2_3_two_wildcards': '([23456789TQKA])\\1{1}[23456789TQKA]JJ'}, 'four_of_a_kind': {'regular': '([23456789TQKA])\\1{3}', 'one_wildcard': '([23456789TQKA])\\1{2}[23456789TQKA]?J', 'two_wildcards': '([23456789TQKA])\\1{1}[23456789TQKA]?JJ', 'three_wildcards': '[23456789TQKA]JJJ'}, 'five_of_a_kind': {'regular': '([

Your puzzle answer was 254494947.

Both parts of this puzzle are complete! They provide two gold stars: **

I know Pain now.